 ## Assignment 3

 This assignment is aimed to give you some practice using Jupyter Notebooks, R, and interpretting statistical output using real-world data. The notebook below will be used to generate the statistical output and you will also write up answers to specific questions related to the statistical output. The assignment will be guided and much of the R code will be provided for you, but specific aspects of the R code you will be asked to interact with and ultimately make a decision about appropriate values to include. The notebook should be run from the first code cell in sequential order, this means that you must run the beginning cells in order to be able to have access to the R packages needed for the assignment and that the data are read in appropriately.

 You may work in groups of up to 3 to complete the assignment. In these situations, please turn in one assignment in ICON with all group members included on the submission.

 *Assignment 3 Due*: **Monday, November 13th, by 11:59 pm**

## Description of the Data

These data are medical charge data from [Kaggle](https://www.kaggle.com/datasets/mirichoi0218/insurance). These data contain information on the medical charges plus individual demographic information about the person associated with those charges. The goal of this analysis will be to try to use the demographic information to try to predict the medical charges for that individual. 

+ **age**: The age of the individual.
+ **sex**: The sex of the individual at birth, male or female. 
+ **bmi**: The body mass index of the individual
+ **children**: A count of the number of children the individual has.
+ **smoker**: Whether the individual smokes or not, yes = smokes, no = does not smoke.
+ **region**: The region of the country the person is located, northeast, northwest, southeast, or southwest.
+ **charges**: The medical insurance charges.

Please don't hesitate to reach out with any data questions about the structure and interpretation of the variables in the data.

### Code Assistance

Furthermore, please reach out if there are specific questions regarding the code below. I do not want the code to be a barrier, but it does allow us to do more interesting things and work with more interesting data. We will provide any support to fix any errors you may encounter with the code. 

## Assignment Setup

**Run this cell first upon opening the notebook everytime** This cell loads the R packages and prepares the data for you.

In [ ]:
library(tidyverse)
library(ggformula)
library(mosaic)
library(rpart)
library(rpart.plot)
library(rsample)

options(scipen = 999)

theme_set(theme_bw(base_size = 14))

charges <- readr::read_csv("https://raw.githubusercontent.com/lebebr01/psqf_4143/main/data/charges.csv")

set.seed(202020)
charges_split <- initial_split(charges, prop = .75)
charges_train <- training(charges_split)
charges_test <- testing(charges_split)

head(charges_test)

## Question 1
 
For the remainder of the assignment, we'll explore the `charges` attribute, which the amount the medical procedure cost in $. Pick another **continuous, quantitative** attribute to complete the code to create a scatterplot of the relationship between `charges` and the attribute you pick. Add in the **continuous, quantitative** attribute of your choosing in place of "$$" below and add appropriate axis labels in place of "@@".

Examples of continuous outcomes to use in place of "$$" include: `age` or `bmi`

Note: The dashed line is forcing the relationship to be linear (a straight line) where as the solid light blue line allows the relationship to be flexible. 

In [ ]:
gf_point(charges ~ $$, data = charges_train, size = 3) |>
  gf_smooth(method = 'lm', linetype = 2, size = 1) |>
  gf_smooth(method = 'loess', linetype = 1, size = 1, color = 'lightblue') |>
  gf_labs(x = "@@",
          y = "@@")

## Question 2

Complete the code below that computes the correlations between the `charges` attribute and a few other attributes. The output should show multiple correlations (one for each row) for each unique value of the `region` attribute. That is, this question is exploring if the relationship depicted in question 1 changes for different regions of the United States. The regions are broken up into 4 regions, northeast, northwest, southeast, and southwest.

Use the same attribute from question #1 in place of "$$". You should have four unique correlations to interpret in the output below.

In [ ]:
charges_train |>
  group_by(region) |>
  summarise(cor = cor(charges ~ $$), 
            num_data = n()) |>
  filter(num_data > 10)

 ## Question 3
 Now fit a regression tree to the song training data to predict the charges for the medical procedure. Complete the code below to add data attributes that you feel will help to predict the medical charges in place of "$$". Note, separate variables with a `+` symbol and include 1 attribute from each of the categories highlighted below.

Include each of the following attributes in the regression tree model. You should have a total of 3 attributes in the model.

 + **Body Mass**: `bmi`
 + **Demographics**: `sex` or `children`

In [ ]:
charges_pop <- rpart(charges ~ $$, data = charges_train, method = 'anova')

rpart.plot(charges_pop, roundint = FALSE, type = 3, branch = .3)

## Question 4
 
The following code to generates the prediction accuracy on the withheld test data. The minimum, maximum, mean, median, standard deviation, and IQR are returned for you.

Note, the command below requires that the model from question #3 is fitted prior to running the code below. If an error occurs, please rerun all cells before this one. 

In [ ]:
charges_test <- charges_test |>
  mutate(pop_predicted = predict(charges_pop, newdata = charges_test),
         error = charges - pop_predicted,
         absolute_error = abs(error))

charges_test |>
  df_stats(~ absolute_error, min, max, mean, median, sd, IQR)

## Question 5

The following code generates conditional error for 6 ranges of medical charges. The resulting figure shows the minimum and maximum error shown by the horizontal line and the average error as a single point along the horizontal line.

*Note*, the command below requires that the model from question #3 and the code from question #4 are processed prior to running the code below. If an error occurs, please rerun all cells before this one. 

**Interpreting y-axis values**  
The values on the y-axis represent the range of medical charges for which the errors are computed for. That is, the y-axis value of (1.33e+04, 2.2e+04] means that the data for this row of the figure include any medical charges that range between $13,300 and $22,000. The e+04 is scientific notation that means move the decimal place four unit to the right. 

In [ ]:
cond_error <- charges_test |>
  df_stats(absolute_error ~ cut_number(charges, 6), mean, min, max) |>
  rename(charges_pop = `cut_number(charges, 6)`)  |>
  mutate(charges_pop2 = gsub("]", "", str_split_fixed(as.character(charges_pop), ",", n = 2)[,2]),
          charges_pop_val = as.numeric(charges_pop2),
          charges_pop2 = forcats::fct_reorder(charges_pop2, charges_pop_val))

gf_pointrangeh(charges_pop ~ mean + min + max, data = cond_error) |>
  gf_labs(y = "Medical Charges", 
                 x = "Absolute Error")

# Question 6

The following code fits a model that includes many more attributes to try to predict the medical charges. The same conditional error plot shown in question #5 is created for this new model with more attribute. The final regression tree output is also shown along with the descriptive absolute error statistics that can be compared with question #4.

*Note*, this code requires the code from questions #3 through #5 to be able to run the code below. 

In the figure, model 1 is the model from question 4, model 2 is the model from question 6. 

**Interpreting y-axis values**  
The values on the y-axis represent the range of medical charges for which the errors are computed for. That is, the y-axis value of (1.33e+04, 2.2e+04] means that the data for this row of the figure include any medical charges that range between $13,300 and $22,000. The e+04 is scientific notation that means move the decimal place four unit to the right. 

In [ ]:
charges_pop_2 <- rpart(charges ~ ., 
      data = charges_train, method = 'anova')

rpart.plot(charges_pop_2, roundint = FALSE, type = 3, branch = .3)

charges_test <- charges_test |>
  mutate(pop_predicted_2 = predict(charges_pop_2, newdata = charges_test),
         error_2 = charges - pop_predicted_2,
         absolute_error = abs(error_2))

charges_test |>
  df_stats(~ absolute_error, min, max, mean, median, sd, IQR)

cond_error_2 <- charges_test |>
  df_stats(absolute_error ~ cut_number(charges, 6), mean, min, max) |>
  rename(charges_pop = `cut_number(charges, 6)`)  |> 
  mutate(charges_pop2 = gsub("]", "", str_split_fixed(as.character(charges_pop), ",", n = 2)[,2]),
          charges_pop_val = as.numeric(charges_pop2),
          charges_pop2 = forcats::fct_reorder(charges_pop2, charges_pop_val),
         model = 'Model 2')

cond_error_comb <- dplyr::bind_rows(mutate(cond_error, model = "Model 1"), 
      cond_error_2)

gf_pointrangeh(charges_pop ~ mean + min + max, data = cond_error_comb, 
     color = ~ model, position = position_dodge(width = .9)) |>
  gf_labs(y = "Medical Charges", 
                 x = "Absolute Error")